## Spatial prediction of soil pollutants with multi-output Gaussian processes
Source: https://nextjournal.com/essicolo/spatial-prediction-of-soil-pollutants-with-multi-output-gaussian-processes?version=latest

In [40]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
np.random.seed(123)

In [41]:
print(pm.__version__)

3.11.4


In [42]:
meuse_df = pd.read_csv("../data/meuse.csv")
meuse_df.head()

,x,y,cadmium,copper,lead,zinc,elev,dist,om,ffreq,soil,lime,landuse,dist.m
0,181072,333611,11.7,85,299,1022,7.909,0.001358,13.6,1,1,1,Ah,50
1,181025,333558,8.6,81,277,1141,6.983,0.012224,14.0,1,1,1,Ah,30
2,181165,333537,6.5,68,199,640,7.800,0.103029,13.0,1,1,1,Ah,150
3,181298,333484,2.6,81,116,257,7.655,0.190094,8.0,1,2,0,Ga,270
4,181307,333330,2.8,48,117,269,7.480,0.277090,8.7,1,2,0,Ah,380


In [43]:
obs_id = meuse_df.index
n_train = np.round(obs_id.shape[0] * 0.7, 0).astype("int")
id_train = np.random.choice(obs_id, size = n_train, replace = False)
id_test = obs_id[~obs_id.isin(id_train)].values

In [44]:
len(id_train), len(id_test)

(108, 47)

In [45]:
targets = ["cadmium", "copper", "lead", "zinc"]
features = ["x", "y", "dist"]

In [46]:
XY = meuse_df[targets + features]

In [47]:
mean_sc = XY.loc[XY.index.isin(id_train), :].mean(axis = 0)
std_sc = XY.loc[XY.index.isin(id_train), :].std(axis = 0)
XY_sc = XY.apply(lambda x: (x-mean_sc)/std_sc, axis = 1)

In [48]:
XY.shape, XY_sc.shape

((155, 7), (155, 7))

In [49]:
XY_sc

,cadmium,copper,lead,zinc,x,y,dist
0,2.341026,1.897420,1.265781,1.432640,1.335194,1.794602,-1.173337
1,1.467739,1.724563,1.071949,1.749590,1.274062,1.744386,-1.116452
2,0.876157,1.162780,0.384727,0.415202,1.456158,1.724489,-0.641089
3,-0.222495,1.724563,-0.346548,-0.604899,1.629150,1.674274,-0.185304
4,-0.166154,0.298498,-0.337737,-0.572937,1.640856,1.528363,0.270120
...,...,...,...,...,...,...,...
150,-0.729566,-0.911498,-0.936854,-0.847273,-1.120508,-1.454268,1.038340
151,-0.081642,-0.090430,0.155653,0.031665,-1.249276,-1.350046,1.038340
152,-0.363348,-0.436143,-0.320116,-0.378506,-1.522421,-1.332044,0.270120
153,-0.729566,-0.868284,-0.919233,-0.857926,-0.753714,-1.265722,0.696856


In [50]:
XY_m = XY_sc.reset_index().melt(id_vars = ["index"] + features, value_vars = targets)
XY_m.sample(8)

,index,x,y,dist,variable,value
182,27,1.206426,0.919141,0.497759,copper,-0.738641
318,8,1.319586,1.434564,-0.213992,lead,-0.196769
198,43,0.474138,0.297602,0.497748,copper,-0.868284
588,123,0.285538,0.549628,-0.926609,zinc,0.926584
444,134,-0.167102,-0.371311,1.152165,lead,-0.998528
438,128,1.062049,1.325605,-0.755263,lead,-0.064611
74,74,-1.318213,-1.015590,-0.128337,cadmium,-0.137984
55,55,-0.466261,0.420773,-1.152009,cadmium,1.355056


In [51]:
XY_m.shape

(620, 6)

In [52]:
variable_ids = pd.DataFrame(dict(
    variable = targets,
    variable_id = np.arange(0, len(targets))
))
variable_ids

,variable,variable_id
0,cadmium,0
1,copper,1
2,lead,2
3,zinc,3


In [53]:
XY_m = XY_m.merge(variable_ids, on = "variable")
XY_m.sample(8)

,index,x,y,dist,variable,value,variable_id
236,81,0.137259,-1.292251,-0.898948,copper,1.508493,1
18,18,0.753786,1.070736,-1.180446,cadmium,1.495909,0
211,56,-0.329689,0.301392,-0.413110,copper,-0.220072,1
221,66,-0.748511,-0.560805,-0.071432,copper,1.465279,1
73,73,-1.216759,-0.908526,-0.128337,cadmium,-0.081642,0
357,47,0.307649,0.136532,1.038283,lead,-0.179148,2
53,53,-0.094264,0.509835,-1.152009,cadmium,2.425538,0
359,49,0.199692,-0.119285,1.977437,lead,0.058737,2


### Create the model

In [54]:
["variable_id"] + features

['variable_id', 'x', 'y', 'dist']

In [55]:
X_mod = XY_m.loc[XY_m["index"].isin(id_train), ["variable_id"] + features].values
id_col = 0
n_mod = n_train
y_mod = XY_m.loc[XY_m["index"].isin(id_train), "value"].values

In [56]:
X_mod.shape, y_mod.shape, n_mod, n_mod*4

((432, 4), (432,), 108, 432)

In [58]:
id_col

0

In [59]:
with pm.Model() as model:
    # GP
    ## Feature covariance
    length_scale = pm.HalfCauchy("length_scale", beta = 10)
    amplitude = pm.HalfCauchy("amplitude", beta = 10)
    cov_feature = amplitude ** 2 * pm.gp.cov.Matern52(
        input_dim = X_mod.shape[1],
        ls = length_scale,
        active_dims = np.arange(1, X_mod.shape[1]) # all except index
    )

    ## Coregion covariance
    W = pm.Normal(
        "W", mu = 0, sd = 5,
        shape = (n_mod, n_mod)
    )
    kappa = pm.HalfCauchy("kappa", beta = 10, shape = n_mod)
    coreg = pm.gp.cov.Coregion(
        input_dim = X_mod.shape[1],
        active_dims = [id_col], # only index
        kappa = kappa,
        W = W
    )

    ## Combined covariance
    cov_f = coreg * cov_feature

    ## GP noise
    σ = pm.HalfCauchy("σ", beta = 10)

    ## Gaussian process
    gp = pm.gp.Marginal(cov_func = cov_f)

    ## Marginal likelihood
    y_ = gp.marginal_likelihood("y", X = X_mod, y = y_mod, noise = σ)

In [60]:
coreg.B.owner.inputs

[dot.0, AllocDiag{offset=0, axis1=0, axis2=1}.0]

In [61]:
cov_feature

In [62]:
coreg.W

W ~ Normal

In [63]:
%%time
with model:
    # Fit
    # trace = pm.sample(2000, tune = 500, chains = 4, cores = 4, return_inferencedata = True) # long
    mp = pm.find_MAP() # quick


CPU times: user 14.3 s, sys: 11.5 s, total: 25.8 s
Wall time: 4.58 s


In [64]:
%%time
with model:
    pred_ = gp.conditional("pred_", XY_m[["variable_id"] + features].values)
    pred_samples = pm.sample_posterior_predictive([mp], var_names=["pred_"], samples = 200)

CPU times: user 1min 8s, sys: 46.2 s, total: 1min 54s
Wall time: 15.2 s


In [65]:
pred_samples

{'pred_': array([[ 2.38038787,  1.98626389,  0.7084253 , ..., -0.56707096,
         -1.18848328,  0.02461363],
        [ 2.01980058,  1.58821731,  0.61798865, ...,  0.45129277,
         -1.11394631, -0.45484847],
        [ 1.98645959,  2.05956292,  0.34680478, ..., -0.09792503,
         -0.74530485, -0.43835727],
        ...,
        [ 1.99734253,  1.96951582,  1.37579278, ..., -0.20442146,
         -1.41400666,  0.2522819 ],
        [ 1.54553537,  1.36936326, -0.44869083, ..., -0.12507816,
         -0.70986375,  0.02783579],
        [ 1.63923958,  1.61366226,  0.9862756 , ..., -0.57483491,
         -1.1130442 , -0.41851767]])}

In [ ]:
### Check Kron_dot

In [68]:
with pm.Model() as model:
    # GP
    ## Feature covariance
    length_scale = pm.HalfCauchy("length_scale", beta = 10)
    amplitude = pm.HalfCauchy("amplitude", beta = 10)
    cov_feature = amplitude ** 2 * pm.gp.cov.Matern52(
        input_dim = X_mod.shape[1],
        ls = length_scale,
        active_dims = np.arange(1, X_mod.shape[1]) # all except index
    )

    ## Coregion covariance
    W = pm.Normal(
        "W", mu = 0, sd = 5,
        shape = (n_mod, n_mod)
    )
    kappa = pm.HalfCauchy("kappa", beta = 10, shape = n_mod)
    coreg = pm.gp.cov.Coregion(
        input_dim = X_mod.shape[1],
        active_dims = [id_col], # only index
        kappa = kappa,
        W = W
    )

    ## Combined covariance
    cov_f = pm.math.kron_dot(coreg, cov_feature) #coreg ⊗ cov_feature

    ## GP noise
    σ = pm.HalfCauchy("σ", beta = 10)

    ## Gaussian process
    gp = pm.gp.Marginal(cov_func = cov_f)

    ## Marginal likelihood
    y_ = gp.marginal_likelihood("y", X = X_mod, y = y_mod, noise = σ)

AttributeError: 'Prod' object has no attribute 'ndim'